### MATH2504 S2, 2022, BigHW Submission 
### Submission form IV of IV 

---

**Note: It is expected that this notebook be runnable in the sense that the marker can select "Restart & Run All" and all code cells of solutions will execute. This requires not to have name clashes between solutions.**

---

# Solution to [Question 13](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q13)

## 13a:

In [ ]:
using Plots
x1 = 1/2
x2 = 1
x3 = 2

f1(y) = sin(y^2)
dydx1 = 2x1*cos(x1^2)

f2(y) = exp(y)
dydx2 = exp(x2)

f3(y) = atan(y) / (1+exp(-y^2))
dydx3 = exp(x3^2)*((2x3^3+2x3)*atan(x3)+exp(x3^2)+1)/((exp(x3^2)+1)^2*(x3^2+1))

for_diff_matrix1 = Vector{Float64}(undef, 100)
cent_diff_matrix1 = Vector{Float64}(undef, 100)

for_diff_matrix2 = Vector{Float64}(undef, 100)
cent_diff_matrix2 = Vector{Float64}(undef, 100)

for_diff_matrix3 = Vector{Float64}(undef, 100)
cent_diff_matrix3 = Vector{Float64}(undef, 100)

h = 10 .^ range(-10, stop=0, length=100)

for n in 1:length(h)
    forward_diff1 = (f1(x1+h[n])-f1(x1))/h[n]
    forward_error1 = abs(forward_diff1-dydx1)
    for_diff_matrix1[n] = forward_error1
    central_diff1 = (f1(x1+h[n]/2)-f1(x1-h[n]/2))/h[n]
    central_error1 = abs(central_diff1-dydx1)
    cent_diff_matrix1[n] = central_error1

    forward_diff2 = (f2(x2+h[n])-f2(x2))/h[n]
    forward_error2 = abs(forward_diff2-dydx2)
    for_diff_matrix2[n] = forward_error2
    central_diff2 = (f2(x2+h[n]/2)-f2(x2-h[n]/2))/h[n]
    central_error2 = abs(central_diff2-dydx2)
    cent_diff_matrix2[n] = central_error2

    forward_diff3 = (f3(x3+h[n])-f3(x3))/h[n]
    forward_error3 = abs(forward_diff3-dydx3)
    for_diff_matrix3[n] = forward_error3
    central_diff3 = (f3(x3+h[n]/2)-f3(x3-h[n]/2))/h[n]
    central_error3 = abs(central_diff3-dydx3)
    cent_diff_matrix3[n] = central_error3
end

#Finds the optimal h for each function, and their forward and central differene schemes
optimal_h = [getindex(minimum(for_diff_matrix1)), getindex(minimum(cent_diff_matrix1)), 
    getindex(minimum(for_diff_matrix2)), getindex(minimum(cent_diff_matrix2)), 
    getindex(minimum(for_diff_matrix3)), getindex(minimum(cent_diff_matrix3))]

@show optimal_h

plot(h, for_diff_matrix1, xaxis = :log10, yaxis = :log10, title = "Step Size vs Error", label = "f1, Forward")
plot!(h, cent_diff_matrix1, label = "f1, Central")
plot!(h, for_diff_matrix2, label = "f2, Forward")
plot!(h, cent_diff_matrix2, label = "f2, Central")
plot!(h, for_diff_matrix3, label = "f3, Forward")
plot!(h, cent_diff_matrix3, label = "f3, Central")
xlabel!("Step Size (h)")
ylabel!("Error")

## 13b:

For $K$ dimensions, we need to calculate $K^2$ partial derivatives. 

Consider the definition of forward difference scheme for partial derivatives:

$\frac{\partial S_i}{\partial z_j} = \lim_{h\rightarrow 0}\frac{S_i(z_1,\dots,z_j+h,\dots, z_K)-S_i(z_1,\dots, z_K)}{h}$

Here we will need to do $K^2$ evaluations to calculate $S_i(z_1,\dots,z_j+h,\dots,z_K)$ for each $1\leq i,j\leq K$. Also, we need one more calculation for $S_i(z_1,\dots,z_K)$, so overall we have $K^2+1$ softmax evaluations for the forward difference scheme.

Consider the central difference scheme:

$\frac{\partial S_i}{\partial z_j} = \lim_{h\rightarrow 0}\frac{S_i(z_1,\dots,z_j+h/2,\dots, z_K)-S_i(z_1,\dots,z_j-h/2,\dots,z_K)}{h}$

Here, we need $K^2$ evaluations for each $S_i(z_1,\dots,z_j+h/2,\dots,z_K)$, and another $K^2$ exaluations for each $S_i(z_1,\dots,z_j-h/2,\dots,z_K)$. So, overall we need $2K^2$ softmax evaluations for the central difference scheme.

## 13c:

In [ ]:
using LinearAlgebra
using Plots
function soft_max_function(z)
    S = exp.(z)/sum(exp.(z))
    return S
end

function forward_softmax(z,h,K)
    forward_J = Matrix{Float64}(undef, K, K)
    softmax = soft_max_function(z)
    for i in 1:K
        z_plus_h = copy(z)
        z_plus_h[i] += h
        forward_J[:, i] = (soft_max_function(z_plus_h) - softmax)/h
    end
    return forward_J
end

function central_softmax(z,h,K)
    central_J = Matrix{Float64}(undef, K, K)
    for i in 1:K 
        z_plus_h,  z_minus_h = copy(z), copy(z)
        z_plus_h[i] += h
        z_minus_h[i] += -h
        central_J[:, i] = (soft_max_function(z_plus_h) - soft_max_function(z_minus_h))/(2*h)
    end
    return central_J
end

function jacobian_softmax(K,z)
    soft = soft_max_function(z)
    J = Matrix{Float64}(undef, K, K)
    for i in 1:K
        for j in 1:K
            J[i,j] = (i == j) ? soft[i] * (1 - soft[i]) : - soft[i] * soft[j]
        end
    end
    return J
end

function soft_max_plotter(K)
    z = [i^(1/3) for i in 1:K]
    h = [1/(10^(n)) for n  in 1:0.1:10]
    J = jacobian_softmax(K,z)
  
    forward_error = [norm(J - forward_softmax(z,h[i],K)) for i in eachindex(h) ]
    central_error = [norm( J - central_softmax(z,h[i],K)) for i in eachindex(h) ]

    optF = getindex(minimum(forward_error))
    optC = getindex(minimum(central_error))

    p1 = plot(h, forward_error, xaxis = :log10, yaxis = :log10, label = "Forward Diff, Optimal Step Size ≈ $optF ", legend = :topleft,
    xlabel = "Step size", ylabel = "Euclidean error", title = "Jacobian Softmax Error, K = $K")
    plot!(h, central_error, xaxis = :log10, yaxis = :log10, label = "Central Diff, Optimal Step Size ≈ $optC")
    return p1
end
p1 = soft_max_plotter(10)
p2 = soft_max_plotter(20)
p3 = soft_max_plotter(100)

softplot = plot(p1,p2,p3, layout = (1,3), size = (1500,500))

# Solution to [Question 14](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q14)

In [ ]:
using LinearAlgebra
using StatsBase

function structured_P(L::Int; p::Float64 = 0.45, r::Float64 = 0.01)::Matrix{Float64}
    q = 1 - p - r
    P = diagm(fill(r,L)) + diagm(-1=>fill(q,L-1)) + diagm(1 => fill(p,L-1))
    P[1,1] = 1-p
    P[L,L] = 1-q
    return P
end

structured_π(L::Int; p::Float64 = 0.45, r::Float64 = 0.01)::Vector{Float64} = begin
    q = 1 - p - r
    [(p/q)^i  for i in 1:L] * (q-p) / p / (1-(p/q)^L) #Explicit expression (birth death)
end;

#Function for method 1, πP=P
function markov_chains_1(P::Matrix{Float64})
    rows, cols = size(P)
    @assert rows == cols #Asserts that P is a square matrix
    A = zeros(1, cols)
    A1 = hcat(A, [1])
    B = ones(Float64, cols, 1)
    P1 = P-I
    P2 = hcat(P1, B)

    π = A1 / P2

    return transpose(π)
end

#Function for method 2
function markov_chains_2(P::Matrix{Float64})
    rows, cols = size(P)
    @assert rows == cols #Assert that P is a square matrix
    π = Vector{Float64}(undef, cols) #Defines empty vector with equal number of columns as P
    
    n = 100
    lim_P = P^n 
    π = lim_P[1,:] #Choose first row since all rows are identical
    return π
end

#Function for method 3
function markov_chains_3(P::Matrix{Float64})
    rows, cols = size(P)
    @assert rows == cols #Assert that P is a square matrix

    P_eigvecs = eigvecs(P) #Find the eigenvectors of P. 
    #eigvals() lists all eigenvalues in ascending order, and eigvecs() lists eigenvectors corresponding to each eigenvalue in the same order. 
    #If 1 is the maximal eigenvalue, then the last vector in eigvecs() will be the corresponding eigenvector.
    λx = P_eigvecs[:,cols]
    new_λx = [abs(λx[i,1]) for i ∈ 1:cols]
    π = new_λx / sum(new_λx)
    sum(π)
    return π
end

#Function for method 4
function markov_chains_4(P::Matrix{Float64})
    rows, cols = size(P)
    @assert rows == cols #Assert that P is a square matrix

    N = 100
    states = 1:cols
    X = sample(states, N, replace=true)
    #indicator = Vector{Int8}(undef, cols)
    #indicator = 0 * indicator
    indicator = zeros(cols,1)
    
    for i ∈ 1:cols
        for n ∈ 1:N
            ((X[n] == i) ? indicator[i] += 1 : indicator[i] += 0)
        end  
    end
    π = (1/N) * indicator
   return π 
end

L = [2,3,4,5,10, 20, 30, 40, 50, 100, 200, 300, 400, 500, 1000]
error1 = Vector{Float64}(undef, length(L))
error2 = Vector{Float64}(undef, length(L))
error3 = Vector{Float64}(undef, length(L))
error4 = Vector{Float64}(undef, length(L))

for i ∈ length(L)
    P = structured_P(L[i])
    π = structured_π(L[i])

    π1 = markov_chains_1(P)
    π2 = markov_chains_2(P)
    π3 = markov_chains_3(P)
    π4 = markov_chains_4(P)

    error1[i] = norm(π - π1)
    error2[i] = norm(π - π2)
    error3[i] = norm(π - π3)
    error4[i] = norm(π - π4)
end

plot(L, error1, title = "Error vs Number of States", label = "Method 1", legend = :topleft)
plot!(L, error2, label = "Method 2")
plot!(L, error3, label = "Method 3")
plot!(L, error4, label = "Method 4")
xlabel!("Number of States")
ylabel!("Error")

# Solution to [Question 15](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q15)

In [ ]:
state_v_x(u::Vector{Float64}) = u[1]
state_v_y(u::Vector{Float64}) = u[2]
state_x(u::Vector{Float64}) = u[3]
state_y(u::Vector{Float64}) = u[4]

function df_dt_one_body(u::Vector{Float64}; t = 0)::Vector{Float64}
    M, G = 1, 1 #We take these constants as normalized. Naturally they would need to be set for physical values.
    r = sqrt(state_x(u)^2 + state_y(u)^2)
    return [-M*G*state_x(u)/r^3, -M*G*state_y(u)/r^3, state_v_x(u), state_v_y(u)]
end;

using Plots, Measures
function trajectory_plotter( t::AbstractArray{T}, 
                        u::Vector{Vector{Float64}}; 
                        title::String = "",
                        label::Union{String, Bool} = false) where T
    x, y, v_x, v_y = state_x.(u), state_y.(u), state_v_x.(u), state_v_y.(u)
 

    p1 = plot(  x, y, label = label, xlabel= "X", ylabel = "Y",
                title = title, aspectratio=1,legend=:topleft,ylim=(-7,7))
    scatter!([0], [0], ms=15, msw=0, c=:orange, shape =:star, label="Sun")
    scatter!([x[1]], [y[1]], ms=5, msw=0, c=:blue, shape =:circle, label="Earth initial position") ;
    return p1
end;

function energy_plotter( t::AbstractArray{T}, 
    infos::Vector{Vector{Vector{Float64}}}; 
    title::String = "", lbl) where T
    p = plot(xlabel = "Time", ylabel = "Energy", title = title*" (energy)", margin = 10mm,size=(800,400))

    for i in 1:length(infos)
        u = infos[i]
        x, y, v_x, v_y = state_x.(u), state_y.(u), state_v_x.(u), state_v_y.(u)
        r = @. sqrt(x^2 + y^2)
        E = @. 0.5*(v_x^2 + v_y^2) - 1.0/r
        plot!(t[i], E, label = lbl[i]) 
    end
    return p
end;

function euler_orbit_solver(h::Float64, t, u_0::Vector{Float64})
    V = [Vector{Float64}(undef,4) for _ in 1:length(t)]
    V[1] = u_0
    temp = [V[i] = V[i-1] + h * df_dt_one_body(V[i-1]) for i in 2:length(t)]
    out = [V[i] for i in 1:length(t)]
    return out
end

function rk_solver(h::Float64, t, u_0::Vector{Float64})
    V = [Vector{Float64}(undef,4) for _ in 1:length(t)]
    V[1] = u_0
    for i in 2:length(t)
        k1 = df_dt_one_body(V[i-1])
        k2 = df_dt_one_body(V[i-1] + h/2 * k1)
        k3 = df_dt_one_body(V[i-1] + h/2 * k2)
        k4 = df_dt_one_body(V[i-1] + h* k3)
        V[i] = V[i-1] + h/6 * (k1 + 2*k2 + 2*k3 + k4)
    end
    return V
end;
function two_trajectory_animator(t1,v1::Vector{Vector{Float64}}
    ,t2,v2::Vector{Vector{Float64}}; 
    step = 200, title::String, label1::String, label2::String)
x1, y1 = state_x.(v1), state_y.(v1)
x2, y2 = state_x.(v2), state_y.(v2)
p = plot(xlabel= "X", ylabel = "Y",
title = title * "at t = 0", aspectratio=1,legend=:topleft,ylim=(-7,7), xlim = (-10,8))
scatter!([0], [0], ms=15, msw=0, c=:orange, shape =:star, label="Sun")
scatter!([x1[1]],[y1[1]], color = :blue, label = label1, markersize = 4)
scatter!([x2[1]],[y2[1]], colour = :red, label = label2, markersize = 4)
scatter!([x1[1]],[y1[1]], color = :black, label = "Initial position", markersize = 4)

length(t1) > length(t2) ? t = length(t2) : t = length(t1)
@gif for i in 2:t
time = floor(0.01*i)
plot!(x1[(i-1):i], y1[(i-1):i], color = :blue, label = :none, title = "Trajectories at t = $time")
plot!(x2[(i-1):i], y2[(i-1):i], color = :red, label = :none)
end every step
end




## 15a:

In [ ]:


u_0 = [0., 1, 1.5, 0] ;

T = 200 ;

h1 = 0.01 ;
t1 = 1:h1:T ;
p1 = euler_orbit_solver(h1,t1,u_0) ;
g1 = trajectory_plotter(t1,p1,title = "Euler Method (Trajectory)", label="h=$h1") ;

h2 = 0.001 ;
t2 = 1:h2:T ;
p2 = euler_orbit_solver(h2,t2,u_0) ;
g2 = trajectory_plotter(t2,p2,title = "Euler Method (Trajectory)", label="h=$h2") ;

h3 = 0.0001 ;
t3 = 1:h3:T ;
p3 = euler_orbit_solver(h3,t3,u_0) ;
p3corrected = [p3[i] for i = 1:2:length(p3)]
t3corrected = [t3[i] for i = 1:2:length(t3)]
g3 = trajectory_plotter(t3corrected,p3corrected,title = "Euler Method (Trajectory)", label="h=$h3") ;

info = [p1,p2,p3corrected] ;
tvec = [t1,t2,t3corrected] ;
hvec = ["h = 0.01","h = 0.001","h = 0.0001"] ;

g4 = energy_plotter(tvec, info, title = "Euler Method", lbl = hvec) ;

eulerplot = plot(g1,g2,g3,g4, layout = (2,2), size = (1280,960))



## 15b:

In [ ]:


u_0 = [0, 1, 1.5, 0] ;

T = 200 ;

h1 = 0.01 ;
t1 = 1:h1:T ;
p1 = rk_solver(h1,t1,u_0) ;
g1 = trajectory_plotter(t1,p1,title = "Runge-Kutta method (Trajectory)", label="h=$h1") ;

h2 = 0.1 ;
t2 = 1:h2:T ;
p2 = rk_solver(h2,t2,u_0) ;
g2 = trajectory_plotter(t2,p2,title = "Runge-Kutta method (Trajectory)", label="h=$h2") ;

h3 = 0.5 ;
t3 = 1:h3:T ;
p3 = rk_solver(h3,t3,u_0) ;
g3 = trajectory_plotter(t3,p3,title = "Runge-Kutta method (Trajectory)", label="h=$h3") ;

info = [p1,p2,p3] ;
tvec = [t1,t2,t3] ;
hvec = ["h = 0.01","h = 0.1","h = 0.5"] ;

g4 = energy_plotter(tvec, info, title = "Runge-Kutta method", lbl = hvec) ;

rkplot = plot(g1,g2,g3,g4, layout = (2,2), size = (1280,960))



## 15c:

The Runge Kutta method, even with larger step sizes achieves better accuracy in calculating trajectories compared to the Euler Method. The Euler method with $h = 0.01$ has significant deviations from the correct orbit over time, however the Runge Kutta method with $h = 0.5$ maintains reasonably consistent orbits over the time period measured. We can achieve the accuracy of the RK method using the Euler method by decreasing the step size to $h = 0.0001$, however this becomes significantly more computationally intensive, as we have to run iterate over a higher order of magnitude of points. It is worth nothing though that with the same step size, the Euler Method is less computationally intensive due to its simplicity. If speed is critical with accuracy less so, it may be worth using the Euler method. However,the RK method is more accurate and consistent in exchange for the extra calculations.  

## 15d:

In [ ]:


u_0 = [0., 1, 1.5, 0] ;

T = 200 ;

h1 = 0.01 ;
t1 = 1:h1:T ;
p1 = rk_solver(h1,t1,u_0) 

h2 = 0.01 ;
t2 = 1:h1:T ;
p2 = euler_orbit_solver(h2,t2,u_0) ;

trajecanim = two_trajectory_animator(t1,p1,t2,p2, title = "Trajectories", step = 200,  label1 = "Runge Kutta h = 0.01", label2 = "Euler Method h = 0.01")



[Trajectories Animation Link](https://github.com/nhamid289/Nazeef-Hamid__Mikayla-Prentice-2504-2022-BigHW/blob/main/Q15d_trajectories_animation.gif)

# Solution to [Question 16](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q16)

In [ ]:
function leap_frog_solver(h::Float64, t, init::Vector{Float64})
    V = [Vector{Float64}(undef,4) for _ in 1:length(t)]
    #V = [Vector{Float64}(undef,2) for _ in 1:length(t)]
    #U = [Vector{Float64}(undef,2) for _ in 1:length(t)]
    V[1] = init
    for i in 2:length(t)
        vhalf = V[i-1][1:2] + (h/2) * df_dt_one_body(V[i-1])[1:2] # only velocity
        V[i][3:4] = V[i-1][3:4] + h * vhalf # only position
        V[i][1:2] = vhalf + (h/2) * df_dt_one_body(V[i])[1:2] # only velocity
    end
    return V
end

function energy_error(u::Vector{Vector{Float64}})
    x, y, v_x, v_y = state_x.(u), state_y.(u), state_v_x.(u), state_v_y.(u)
    r = @. sqrt(x^2 + y^2)
    E = @. 0.5*(v_x^2 + v_y^2) - 1.0/r
    diff = abs(maximum(E) - minimum(E))
    return diff
end

function planet_system_ODE!(du, u, p, t)
    M, G = 1, 1
    r = sqrt(u[3]^2 + u[4]^2)
    du[1] = - M * G * u[3] / r^3
    du[2] = - M * G * u[4] / r^3
    du[3] = u[1]
    du[4] = u[2]
end

function f1(dv, v, u, p, t)
    r = sqrt(u[1]^2 + u[2]^2)
    dv[1] = -u[1] / r^3
    dv[2] = -u[2] / r^3
end

function f2(du, v, u, p, t)
    du[1] = v[1]
    du[2] = v[2]
end



## 16a:

In [ ]:



u_0 = [0.0,  1.0 , 1.5 , 0.0]

T = 200 ;

h1 = 0.01 ;
t1 = 1:h1:T ;
p1 = leap_frog_solver(h1,t1,u_0) ;
g1 = trajectory_plotter(t1,p1,title = "Leap frog method (Trajectory)", label="h=$h1") ;

h2 = 0.1 ;
t2 = 1:h2:T ;
p2 = leap_frog_solver(h2,t2,u_0) ;
g2 = trajectory_plotter(t2,p2,title = "Leap frog method (Trajectory)", label="h=$h2") ;

h3 = 0.5 ;
t3 = 1:h3:T ;
p3 = leap_frog_solver(h3,t3,u_0) ;
g3 = trajectory_plotter(t3,p3,title = "Leap frog method (Trajectory)", label="h=$h3") ;

info = [p1,p2,p3] ;
tvec = [t1,t2,t3] ;
hvec = ["h = 0.01","h = 0.1","h = 0.5"] ;

g4 = energy_plotter(tvec, info, title = "Leap Frog Method ", lbl = hvec) ;

leapplot = plot(g1,g2,g3,g4, layout = (2,2), size = (1280,960))



## 16b:

We consider a numerical operator such as `+, -, *, / , ^2 ` etc. as a floating point operation. First, we should see how many floating point operations are executed in the `df_dt_one_body()` function. We take the square of two 4 element vectors, then square root the sum. This gives us 10 floating point operations per call of `df_dt_one_body()`. 
Let us first consider the Euler method. Each cycle has a `+` and a `*` operation acting on 4 element vectors, and it has one call of the `df_dt_one_body()` function. This means $8 + 10 = 18$ floating point operations. 
Next, we consider the Runge Kutta method. This has multiple sub iterations for each loop, causing many more operations to be calculated. Calculating `k1` requires a call of `df_dt` (+10 operations). For `k2`, `k3` , each call `df_dt` and each have two operations on 4 element vectors, as well as a single operation acting on the scalar `h`. (+19 $\times$ 2). `k4` has a call for `df_dt` and two operations on 4 element vectors (+18). The final line has an additional 29 operations. In total we see that there are 95 floating point operations per loop. 
Finally, let us examine the Leapfrog method. Note that for the sub iterations in the leapfrog method, we only update 2 elements of the 4 element vector for any operation. With this in mind, using a similar process used for the Runge Kutta process, we find that the leapfrog method has 24 floating point operations.

From this analysis we can conclude that at the same time step, the Runge Kutta method is the most computationally intensive, while the Euler method is the least. We should note that the Runge Kutta method produces significantly better results the the Euler method. However, undoubtedly from an computational expense per performance perspective, the leapfrog method is superior, being only slightly more expensive than the Euler method but delivering performance closer to the Runge Kutta method. 

## 16c:

In [ ]:

times = [10^(5/n) for n  in 1:100]

diffEuler = Vector{Float64}(undef,length(times))
diffRK = Vector{Float64}(undef,length(times))
diffLeap = Vector{Float64}(undef,length(times))

for i in 1:length(times)
    init = [0., 1, 1.5, 0] 
    h = 0.05
    t = 1:h:(times[i])
    e = euler_orbit_solver(h,t,init)
    r = rk_solver(h,t,init)
    l = leap_frog_solver(h,t,init)
    diffEuler[i] = energy_error(e)
    diffRK[i] = energy_error(r)
    diffLeap[i] = energy_error(l)
end

error_energy_graphs = plot(xlabel = "Maximum Time", ylabel = "Energy Error", title = "System energy change for varying time horizons", xaxis = :log, yaxis = :log,
xticks = [1, 10, 10^2, 10^3, 10^4, 10^5], legend = :bottomright)
    plot!(times, diffEuler, label = "Euler Method ") ;
    plot!(times, diffRK, label = "Runge Kutta Method") ;
    plot!(times, diffLeap, label = "Leapfrog Method") 


## 16d:

In [ ]:
using DifferentialEquations



T = 200.

u_0 = [0., 1, 1.5, 0] 
tspan = (0.0,T)
prob = ODEProblem(planet_system_ODE!,u_0,tspan)
tsitsol = solve(prob, Tsit5(), dt = 0.01);

tsitp = [vcat(tsitsol[1,i], tsitsol[2,i], tsitsol[3,i], tsitsol[4,i]) for i in 1:length(tsitsol.t[:])]

tsitG = trajectory_plotter(tsitsol.t[:],tsitp,title = "Tsit5 (standard) Trajectory") 
tsitE = energy_plotter([tsitsol.t[:]],[tsitp],title = "Tsit5 (standard)", lbl = ["Tsit5 solver"])



u0 = [1.5, 0]
v0 = [0. , 1]

dynaProb = DynamicalODEProblem(f1,f2,v0, u0, tspan)
yoshidasol = solve(dynaProb, Yoshida6(), dt = 0.5)

yoshidap = [vcat(yoshidasol[1,i], yoshidasol[2,i], yoshidasol[3,i], yoshidasol[4,i]) for i in 1:length(yoshidasol.t[:])];
yoshidaG = trajectory_plotter(yoshidasol.t[:],yoshidap,title = "Yoshida6 (symplectic) Trajectory") ;
yoshidaE = energy_plotter([yoshidasol.t[:]],[yoshidap],title = "Yoshida6 (symplectic)", lbl = "Yoshida6") ;

diffSolversenergy = energy_plotter([tsitsol.t[:], yoshidasol.t[:]], [tsitp, yoshidap], title = "ODE Solvers from package", lbl = ["Tsit5", "Yoshida6"]) ;

diffsolverplots = plot(tsitG, yoshidaG, diffSolversenergy, layout = (3,1), size = (960,1280))

For this question, I chose the `tsit5` and `Yoshida6` solvers. `tsit5` is a standard ODE solver, while `Yoshida6` is a symplectic solver. We see that the standard ODE solver struggles with the planetary motion example, with the trajectory deviating over time, and energy not remaining constant. However, the Yoshida6 symplectic solver excels, even with a step size of 0.5. The documentation for symplectic integrators notes that they are ideal for constant energy systems, and this is clearly represented in the results, with the Yoshida6 energy curve being very constant.

In [ ]:

diffTsit5 = Vector{Float64}(undef,length(times))
diffYoshida6 = Vector{Float64}(undef,length(times))

for i in 1:length(times)
    timespan = (0, times[i])
    local tprob = ODEProblem(planet_system_ODE!,u_0,timespan) ;
    local tsitsol = solve(tprob, Tsit5(), dt = 0.01);
    local tsitp = [vcat(tsitsol[1,i], tsitsol[2,i], tsitsol[3,i], tsitsol[4,i]) for i in 1:length(tsitsol.t[:])] ;
    local dynaProby = DynamicalODEProblem(f1,f2,v0, u0, timespan) ;
    local yoshidasol = solve(dynaProby, Yoshida6(), dt = 0.5) ;
    local yoshidap = [vcat(yoshidasol[1,i], yoshidasol[2,i], yoshidasol[3,i], yoshidasol[4,i]) for i in 1:length(yoshidasol.t[:])];
    diffTsit5[i] = energy_error(tsitp) ;
    diffYoshida6[i] = energy_error(yoshidap) ;
end
error_energy_graphs = plot(xlabel = "Maximum Time", ylabel = "Energy Error", title = "System energy change for varying time horizons", xaxis = :log, yaxis = :log,
xticks = [1, 10, 10^2, 10^3, 10^4, 10^5], yticks = [10^6, 10^4, 10^2, 1, 10^-2, 10^-4, 10^-6, 10^-8, 10^-10, 10^-12],legend = :topleft)
    plot!(times, diffEuler, label = "Euler Method h = 0.01 ") ;
    plot!(times, diffRK, label = "Runge Kutta Method h = 0.01") ;
    plot!(times, diffLeap, label = "Leapfrog Method h = 0.01 ") ;
    plot!(times, diffTsit5, label = "Tsit5 h = 0.01 ") ;
    plot!(times, diffYoshida6, label = " Yoshida6 h =0.5") 


I also compared the energy deviations for large timescales using the new methods. We see that `tsit5` performs poorly for large time scales, introducing huge energy changes. However, `Yoshida6` remains very consistent, with a change in energy of around $10^{-6}$. Like the leapfrog method, this change is independent of the time horizon used, meaning the solver will scale very well for longer and longer simulations. Note that even with a step size of 0.5, we are achieving lower errors than the leapfrog method, speeding up computational time signficantly. 

# Solution to [Question 17](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q17)

Dr. Amy Chan finished her undergraduate degree in 2009 and continued to complete an Honours degree in real analysis and topology optimisation in 2010. In 2013, she became a graduate fellow with CSIRO in image analysis, to which she described her work to be “maths hand in hand with programming.” Amy stated that she had not done much statistics in her undergraduate degree, but predicted that it would be useful to her career. Amy then completed her PhD in image analysis and statistics in 2018. While studying, she realised she did not enjoy research and wanted to pursue a career in data analysis, software development, and statistics. 

After her PhD, Amy started working at Polymathian, a company that “uses a variety of techniques to provide Industrial Mathematics solutions to customers”. Such techniques include mathematical optimisation, discrete event simulation and data analytics. She has held many roles there, starting as a statistician, moving to software development and consulting, and now working in a managerial position. 

According to Dr. Chan, Polymathian uses a variety of tools for the mathematical techniques they implement as well as their software development and managed services. For mathematical optimisation, they use operations research, Gurobi and CPLEX in C++. Python is the main programming language they use at Polymathian, especially for their backend software development, discrete event simulation and data analytics. We are familiar with Python and have used it to program before, but have never heard of Gurobi or CPLEX. Amy also stated that they deliver websites to their customers using Amazon Web Services (AWS) and Docker. These tools were unknown to us prior to the guest lecture. 

Amy strongly recommended applying for vacationer scholarships and internships, stating that it can help get an idea of what we might be looking for in a future job. She also mentioned that it is a good idea to try out different fields – research, academia, private sectors etc. – to help decide what career path we want/don’t want. Additionally, Amy strongly emphasised to not let a lack of domain knowledge stop us from trying since we always have the opportunity to learn. 

A big takeaway from the Perspective Seminar was how applicable mathematics is to the real world. It’s fascinating to find out how many fields require people skilled in mathematical literacy and programming. Overall, we really loved hearing such valuable advice from an experienced and intelligent woman in mathematics such as Dr. Amy Chan, and we feel excited about our future careers.